## Get Data

In [22]:
%matplotlib inline
import matplotlib.pyplot as plt
import fitbit
import json
import requests
import time
from pprint import pprint

# gather_keys_oauth2.py file needs to be in the same directory. 
# also needs to install cherrypy: https://pypi.org/project/CherryPy/
# pip install CherryPy
import gather_keys_oauth2 as Oauth2
import pandas as pd 
import datetime



# YOU NEED TO PUT IN YOUR OWN CLIENT_ID AND CLIENT_SECRET
CLIENT_ID='23948W'
CLIENT_SECRET='250a5113354c041785901ff347416bcf'

## API Authorization

In [119]:
server=Oauth2.OAuth2Server(CLIENT_ID, CLIENT_SECRET)
server.browser_authorize()
ACCESS_TOKEN=str(server.fitbit.client.session.token['access_token'])
REFRESH_TOKEN=str(server.fitbit.client.session.token['refresh_token'])
auth2_client=fitbit.Fitbit(CLIENT_ID,CLIENT_SECRET,oauth2=True,access_token=ACCESS_TOKEN,refresh_token=REFRESH_TOKEN)

[15/Jan/2023:02:51:12] ENGINE Listening for SIGTERM.
[15/Jan/2023:02:51:12] ENGINE Bus STARTING
[15/Jan/2023:02:51:12] ENGINE Set handler for console events.
[15/Jan/2023:02:51:12] ENGINE Started monitor thread 'Autoreloader'.
[15/Jan/2023:02:51:13] ENGINE Serving on http://127.0.0.1:8080
[15/Jan/2023:02:51:13] ENGINE Bus STARTED


127.0.0.1 - - [15/Jan/2023:02:51:27] "GET /?code=092610bc2d2b958130d31379e5c6422c9ba65aeb&state=6onfMHq1eJg4o4SsJqW8Zv8h42TLLj HTTP/1.1" 200 122 "" "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36 Edg/108.0.1462.76"
127.0.0.1 - - [15/Jan/2023:02:51:27] "GET /favicon.ico HTTP/1.1" 200 1406 "http://127.0.0.1:8080/?code=092610bc2d2b958130d31379e5c6422c9ba65aeb&state=6onfMHq1eJg4o4SsJqW8Zv8h42TLLj" "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36 Edg/108.0.1462.76"


[15/Jan/2023:02:51:28] ENGINE Bus STOPPING
[15/Jan/2023:02:51:29] ENGINE HTTP Server cherrypy._cpwsgi_server.CPWSGIServer(('127.0.0.1', 8080)) shut down
[15/Jan/2023:02:51:29] ENGINE Stopped thread 'Autoreloader'.
[15/Jan/2023:02:51:29] ENGINE Removed handler for console events.
[15/Jan/2023:02:51:29] ENGINE Bus STOPPED
[15/Jan/2023:02:51:29] ENGINE Bus EXITING
[15/Jan/2023:02:51:29] ENGINE Waiting for child threads to terminate...
[15/Jan/2023:02:51:29] ENGINE Bus EXITED
[15/Jan/2023:02:51:29] ENGINE Waiting for thread Thread-380.


## 5a.) Get One day of Data

In [4]:
# You will have to modify this 
# depending on when you started to use a fitbit
oneDate = pd.datetime(year = 2022, month = 11, day = 13)

C:\Users\sunny\AppData\Local\Temp\ipykernel_32416\2096318665.py:3: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  oneDate = pd.datetime(year = 2022, month = 11, day = 13)


In [3]:
help(auth2_client.intraday_time_series)

Help on method intraday_time_series in module fitbit.api:

intraday_time_series(resource, base_date='today', detail_level='1min', start_time=None, end_time=None) method of fitbit.api.Fitbit instance
    The intraday time series extends the functionality of the regular time series, but returning data at a
    more granular level for a single day, defaulting to 1 minute intervals. To access this feature, one must
    fill out the Private Support form here (see https://dev.fitbit.com/docs/help/).
    For details on the resources available and more information on how to get access, see:
    
    https://dev.fitbit.com/docs/activity/#get-activity-intraday-time-series



In [93]:
oneDayData = auth2_client.intraday_time_series('activities/heart',
                                               base_date=oneDate,
                                               detail_level='1min')
oneDayData
devices = auth2_client.get_devices()
sleepData = auth2_client.get_sleep(oneDate)

oneDayData2 = auth2_client.intraday_time_series('activities/heart',
                                               base_date=pd.datetime(year = 2023, month = 1, day = 11),
                                               detail_level='1min')
                                               
logactivity = auth2_client.activities(date = oneDate)


C:\Users\sunny\AppData\Local\Temp\ipykernel_32416\3393413633.py:9: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  base_date=pd.datetime(year = 2023, month = 1, day = 11),


In [79]:
#print(oneDayData)
print(devices)
#print(oneDayData2)

####THIS CODE CHUNK GETS HEART RATE DATA MINUTE BY MINUTE#####
time_list = []
val_list = []
for i in oneDayData2['activities-heart-intraday']['dataset']:
    val_list.append(i['value'])
    time_list.append(i['time'])
heartdf = pd.DataFrame({'Heart Rate':val_list,'Time':time_list})
#logactivity

heartdf

heartdf.to_csv('C:/Users/sunny/Documents/1-Work/Extracurricular/stanford/Participant Information/firsttest.csv', 
               columns=['Time','Heart Rate'], header=True, \
               index = False)

[{'battery': 'Medium', 'batteryLevel': 37, 'deviceVersion': 'Inspire 2', 'features': [], 'id': '2136767686', 'lastSyncTime': '2023-01-14T23:31:33.000', 'mac': '9EBC34CD76FE', 'type': 'TRACKER'}, {'deviceVersion': 'Aria Air', 'features': [], 'id': 'F-31-A4C138B1ED92', 'type': 'SCALE'}]


In [120]:
#heartdf

#column = heartdf['Heart Rate']

#print(column[column > 20].count())

#wearPercent = (column[column > 20].count())/1440
#print(wearPercent)


start_lookdate = datetime.date(2022, 12, 1)
end_lookdate = datetime.date(2022,12,15)
delta = datetime.timedelta(days=1)
wearList = []
dateList = []
wearPercent = 0


while (start_lookdate <= end_lookdate):
    heartAPI = auth2_client.intraday_time_series('activities/heart',
                                               base_date=start_lookdate,
                                               detail_level='1min')

    time_list = []
    val_list = []
    for i in heartAPI['activities-heart-intraday']['dataset']:
        val_list.append(i['value'])
        time_list.append(i['time'])
    heartdf = pd.DataFrame({'Heart Rate':val_list,'Time':time_list})
    #logactivity
    column = heartdf['Heart Rate']
    
    wearPercent = (column[column > 20].count())/1440
    print("wear percent: " + str(wearPercent))
    
    dateList.append(start_lookdate.strftime("%m/%d/%Y"))
    wearList.append(wearPercent)
    #list.append(wearPercent)
    
    #heartRate[start_lookdate.strftime("%d")] = wearPercent
    newDate = None
    print(start_lookdate)
    #print(start_lookdate.strftime("%m/%d/%Y"))
    #heartRate
    #df.loc[len(df)] = list
    start_lookdate += delta


heartRate = pd.DataFrame({'Date': dateList, 'WearPercent': wearList})


wear percent: 0.02361111111111111
2022-12-01
wear percent: 0.6666666666666666
2022-12-02
wear percent: 0.9638888888888889
2022-12-03
wear percent: 0.9736111111111111
2022-12-04
wear percent: 0.37430555555555556
2022-12-05
wear percent: 0.0
2022-12-06
wear percent: 0.4701388888888889
2022-12-07
wear percent: 0.39444444444444443
2022-12-08
wear percent: 0.0
2022-12-09
wear percent: 0.0
2022-12-10
wear percent: 0.0
2022-12-11
wear percent: 0.0
2022-12-12
wear percent: 0.0
2022-12-13
wear percent: 0.0
2022-12-14
wear percent: 0.0
2022-12-15


In [121]:
heartRate

,Date,WearPercent
0,12/01/2022,0.023611
1,12/02/2022,0.666667
2,12/03/2022,0.963889
3,12/04/2022,0.973611
4,12/05/2022,0.374306
5,12/06/2022,0.000000
6,12/07/2022,0.470139
7,12/08/2022,0.394444
8,12/09/2022,0.000000
9,12/10/2022,0.000000


In [ ]:
type(logactivity)
#not needed code in this section
oneDate = pd.datetime(year = 2022, month = 11, day = 13)

df = pd.DataFrame(columns=['Date', 'activityCalories', 'caloriesBMR', 'caloriesOut', 'fairlyActiveMins', 'lightlyActiveMins', 'veryActiveMins', 'sedentaryMins', 'steps', 'sleep'])

if True:
    for i in range(1,20, 1):
        newDate = pd.datetime(year= 2022, month = 11, day = i)
        logactivity = auth2_client.activities(date = newDate)
        sleepData = auth2_client.get_sleep(newDate)
        list = []
        list.append(newDate.strftime('%m/%d/%Y'))
        list.append(logactivity['summary']['activityCalories'])
        list.append(logactivity['summary']['caloriesBMR'])
        list.append(logactivity['summary']['caloriesOut'])
        list.append(logactivity['summary']['fairlyActiveMinutes'])
        list.append(logactivity['summary']['lightlyActiveMinutes'])
        list.append(logactivity['summary']['veryActiveMinutes'])
        list.append(logactivity['summary']['sedentaryMinutes'])
        list.append(logactivity['summary']['steps'])
        list.append(sleepData['summary']['totalMinutesAsleep'])
        newDate = None
        print(i)
        df.loc[len(df)] = list

#type(oneDate)




In [103]:
df
df.to_csv('first_test_case.csv', index=True)

In [13]:
# Look at the first 5 rows of the pandas DataFrame
df = pd.DataFrame(columns=['Date', 'activityCalories', 'caloriesBMR', 'caloriesOut', 'fairlyActiveMins', 'lightlyActiveMins', 'veryActiveMins', 'sedentaryMins', 'steps', 'sleep'])
start_lookdate = datetime.date(2022, 12, 1)
end_lookdate = datetime.date(2022,12,15)
delta = datetime.timedelta(days=1)
while (start_lookdate <= end_lookdate):
    logactivity = auth2_client.activities(date = start_lookdate)
    sleepData = auth2_client.get_sleep(start_lookdate)
    list = []
    list.append(start_lookdate.strftime('%m/%d/%Y'))
    list.append(logactivity['summary']['activityCalories'])
    list.append(logactivity['summary']['caloriesBMR'])
    list.append(logactivity['summary']['caloriesOut'])
    list.append(logactivity['summary']['fairlyActiveMinutes'])
    list.append(logactivity['summary']['lightlyActiveMinutes'])
    list.append(logactivity['summary']['veryActiveMinutes'])
    list.append(logactivity['summary']['sedentaryMinutes'])
    list.append(logactivity['summary']['steps'])
    list.append(sleepData['summary']['totalMinutesAsleep'])
    newDate = None
    print(start_lookdate)
    df.loc[len(df)] = list
    start_lookdate += delta

df.to_csv('first_test_case.csv', index=True)

2022-12-01
2022-12-02
2022-12-03
2022-12-04
2022-12-05
2022-12-06
2022-12-07
2022-12-08
2022-12-09
2022-12-10
2022-12-11
2022-12-12
2022-12-13
2022-12-14
2022-12-15


In [4]:
df

,Date,activityCalories,caloriesBMR,caloriesOut,fairlyActiveMins,lightlyActiveMins,veryActiveMins,sedentaryMins,steps,sleep
0,12/03/2022,497,1246,1579,26,158,8,1248,4041,0
1,12/04/2022,37,1246,1268,0,18,0,1422,43,0
2,12/05/2022,288,1246,1449,7,92,0,1341,3509,0
3,12/06/2022,206,1246,1398,0,64,0,1376,2807,0
4,12/07/2022,419,1246,1566,5,92,19,1324,6174,0
5,12/08/2022,176,1246,1370,0,63,0,1377,2144,0
6,12/09/2022,132,1246,1337,4,41,4,1391,1587,0


In [8]:

fit_statsHR = auth2_client.intraday_time_series('activities/heart', base_date=datetime.date(2022, 12, 1), detail_level='1min')
time_list = []
val_list = []
for i in fit_statsHR['activities-heart']['dataset']:
    val_list.append(i['value'])
    time_list.append(i['time'])
heartdf = pd.DataFrame({'Heart Rate':val_list,'Time':time_list})


TypeError: list indices must be integers or slices, not str

In [54]:
#auth2_client.heart(date = datetime.date(2022, 12, 1), data = )

HR_request = requests.get('https://api.fitbit.com/1/user/' + CLIENT_ID + '/activities/heart/date/2023-12-1/1d/1min/time/00:01/23:55.json', headers = {'Authorization': 'Bearer ' + ACCESS_TOKEN})

pprint(HR_request.json()['activities-heart-intraday']['dataset'])
hr2req = requests.get('https://api.fitbit.com/1/user/' + CLIENT_ID + '/activities/heart/date/2022-12-1/1d/1min.json', headers = {'Authorization': 'Bearer ' + ACCESS_TOKEN})

hr2req

KeyError: 'activities-heart-intraday'

In [51]:
#pprint(HR_request.json()['activities-heart-intraday']['dataset'])

fit_statsHR = auth2_client.intraday_time_series('activities/heart', base_date=datetime.date(2023, 1, 10), detail_level='1min')
df = pd.DataFrame(fit_statsHR['activities-heart-intraday']['dataset'])
heartrate = auth2_client.intraday_time_series('activities/heart', base_date = datetime.date(2022, 12, 1), detail_level = '1min')
df
#heartrate = auth2_client.heart(date = str((datetime.datetime.now() - datetime.timedelta(days=1)).strftime("%Y-%m-%d")))
heartrate2 = auth2_client.activities(datetime.date(2022, 12, 1))

#fit_statsHR

{'activities': [],
 'summary': {'activeScore': -1,
  'activityCalories': 32,
  'caloriesBMR': 1246,
  'caloriesOut': 1542,
  'distances': [{'activity': 'total', 'distance': 3.57},
   {'activity': 'tracker', 'distance': 3.57},
   {'activity': 'loggedActivities', 'distance': 0},
   {'activity': 'veryActive', 'distance': 0},
   {'activity': 'moderatelyActive', 'distance': 0},
   {'activity': 'lightlyActive', 'distance': 0.11},
   {'activity': 'sedentaryActive', 'distance': 0}],
  'fairlyActiveMinutes': 0,
  'lightlyActiveMinutes': 15,
  'marginalCalories': 7,
  'sedentaryMinutes': 1425,
  'steps': 8906,
  'veryActiveMinutes': 0}}

In [ ]:
# The first part gets a date in a string format of YYYY-MM-DD
filename = oneDayData['activities-heart'][0]['dateTime'] +'_intradata'

# Export file to csv
df.to_csv(filename + '.csv', index = False)
df.to_excel(filename + '.xlsx', index = False)

## ## 5b.) Get Multiple Days of Data

In [ ]:
# startTime is first date of data that I want. 
# You will need to modify for the date you want your data to start
startTime = pd.datetime(year = 2019, month = 11, day = 27)
endTime = pd.datetime.today().date() - datetime.timedelta(days=1)

In [ ]:
date_list = []
df_list = []
allDates = pd.date_range(start=startTime, end = endTime)

for oneDate in allDates:
    
    oneDate = oneDate.date().strftime("%Y-%m-%d")
    
    oneDayData = auth2_client.intraday_time_series('activities/heart', base_date=oneDate, detail_level='1sec')

    df = pd.DataFrame(oneDayData['activities-heart-intraday']['dataset'])
    
    date_list.append(oneDate)
    
    df_list.append(df)
    
final_df_list = []

for date, df in zip(date_list, df_list):

    if len(df) == 0:
        continue
    
    df.loc[:, 'date'] = pd.to_datetime(date)
    
    final_df_list.append(df)

final_df = pd.concat(final_df_list, axis = 0)

In [ ]:
## Optional Making of the data have more detailed timestamp (day and hour instead of day)
hoursDelta = pd.to_datetime(final_df.loc[:, 'time']).dt.hour.apply(lambda x: datetime.timedelta(hours = x))
minutesDelta = pd.to_datetime(final_df.loc[:, 'time']).dt.minute.apply(lambda x: datetime.timedelta(minutes = x))
secondsDelta = pd.to_datetime(final_df.loc[:, 'time']).dt.second.apply(lambda x: datetime.timedelta(seconds = x))

# Getting the date to also have the time of the day
final_df['date'] = final_df['date'] + hoursDelta + minutesDelta + secondsDelta

In [ ]:
final_df.tail()

In [ ]:
filename = 'all_intradata'
final_df.to_csv(filename + '.csv', index = False)

## 6.) Try to Graph Intraday Data

In [ ]:
# this is bad as time is duplicated over many days fixing the date column will fix the problem
final_df.plot('time', 'value')

In [ ]:
# The code below is not efficient as I call to_datetime twice
hoursDelta = pd.to_datetime(final_df.loc[:, 'time']).dt.hour.apply(lambda x: datetime.timedelta(hours = x))
minutesDelta = pd.to_datetime(final_df.loc[:, 'time']).dt.minute.apply(lambda x: datetime.timedelta(minutes = x))
secondsDelta = pd.to_datetime(final_df.loc[:, 'time']).dt.second.apply(lambda x: datetime.timedelta(seconds = x))


In [ ]:
# Getting the date to also have the time of the day
final_df['date'] = final_df['date'] + hoursDelta + minutesDelta + secondsDelta

In [ ]:
#final_df['temp_value'] = final_df['value'] + random.randint(-2, 2)

In [ ]:
# this fixed the problem.
final_df.plot('date', 'value')
plt.legend('')

In [ ]:
## Looking at a couple days only. 
startDate = pd.datetime(year = 2019, month = 12, day = 24)
lastDate = pd.datetime(year = 2019, month = 12, day = 27)

coupledays_df = final_df.loc[final_df.loc[:, 'date'].between(startDate, lastDate), :]

In [ ]:
coupledays_df

In [ ]:
# Just checking the number of the rows 
coupledays_df.shape

In [ ]:
coupledays_df.plot('date', 'value')
plt.legend('')

In [ ]:
fig, ax = plt.subplots(figsize=(10, 7))

# Taken from: https://stackoverflow.com/questions/16266019/python-pandas-group-datetime-column-into-hour-and-minute-aggregations
times = pd.to_datetime(coupledays_df['date'])
coupledays_df.groupby([times.dt.date,times.dt.hour]).value.mean().plot(ax = ax)

ax.grid(True,
    axis = 'both',
    zorder = 0,
    linestyle = ':',
    color = 'k')
ax.tick_params(axis = 'both', rotation = 45, labelsize = 20)
ax.set_xlabel('Date, Hour', fontsize = 24)
ax.set_ylabel('Heart Rate', fontsize = 24)
fig.tight_layout()
fig.savefig('coupledaysavergedByMin.png', format = 'png', dpi = 300)

## 7.) Resting Heart Rate

In [10]:
# startTime is first date of data that I want. 
# You will need to modify for the date you want your data to start
startTime = pd.datetime(year = 2022, month = 12, day = 1)
endTime = pd.datetime.today().date() - datetime.timedelta(days=1)

C:\Users\sunny\AppData\Local\Temp\ipykernel_36380\2906094424.py:3: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  startTime = pd.datetime(year = 2022, month = 12, day = 1)
C:\Users\sunny\AppData\Local\Temp\ipykernel_36380\2906094424.py:4: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  endTime = pd.datetime.today().date() - datetime.timedelta(days=1)


In [11]:
date_list = []
resting_list = []

allDates = pd.date_range(start=startTime, end = endTime)

for oneDate in allDates:
    
    oneDate = oneDate.date().strftime("%Y-%m-%d")
    
    oneDayData = auth2_client.intraday_time_series('activities/heart', base_date=oneDate, detail_level='1sec')
    
    date_list.append(oneDate)
    
    resting_list.append(oneDayData['activities-heart'][0]['value']['restingHeartRate'])

KeyError: 'restingHeartRate'

In [ ]:
fig, ax = plt.subplots(figsize=(10, 7))

ax.plot(date_list, resting_list )

# This is just making it so there isnt a grid line or text for every point
xtick_list = []
xticklabel_list = []
for index, label in enumerate(ax.get_xticklabels()):
    if index % 5 == 0:
        xticklabel_list.append(label)
        xtick_list.append(index)

ax.grid(True,
    axis = 'both',
    zorder = 0,
    linestyle = ':',
    color = 'k')
ax.tick_params(axis = 'both', labelsize = 20)
ax.set_xticks(xtick_list)
ax.tick_params(axis = 'x', rotation = 90,labelsize = 20)
ax.set_xlim(0, index)
#ax.set_xticklabels(ax.get_xticklabels(),rotation = 45, rotation_mode="anchor", ha = 'right')
ax.set_xlabel('Date', fontsize = 24)
ax.set_ylabel('Resting Heart Rate', fontsize = 24)
fig.tight_layout()
fig.savefig('restingHR_graph.png', format = 'png', dpi = 300)

In [ ]:
resting_df = pd.DataFrame({'date': date_list, 'RHR': resting_list})

In [ ]:
resting_df.head()

## 8.) Get Sleep Data

In [ ]:
startTime = pd.datetime(year = 2020, month = 1, day = 5)
endTime = pd.datetime.today().date() - datetime.timedelta(days=1)

In [ ]:
allDates = pd.date_range(start=startTime, end = endTime)

In [ ]:
date_list = []
df_list = []
stages_df_list = []

allDates = pd.date_range(start=startTime, end = endTime)

for oneDate in allDates:
    
    oneDate = oneDate.date().strftime("%Y-%m-%d")
    
    oneDayData = auth2_client.sleep(date=oneDate)
    
    # get number of minutes for each stage of sleep and such. 
    stages_df = pd.DataFrame(oneDayData['summary'])

    df = pd.DataFrame(oneDayData['sleep'][0]['minuteData'])
    
    date_list.append(oneDate)
    
    df_list.append(df)
    
    stages_df_list.append(stages_df)
    
final_df_list = []

final_stages_df_list = []

for date, df, stages_df in zip(date_list, df_list, stages_df_list):

    if len(df) == 0:
        continue
    
    df.loc[:, 'date'] = pd.to_datetime(date)
    
    stages_df.loc[:, 'date'] = pd.to_datetime(date)
    
    final_df_list.append(df)
    final_stages_df_list.append(stages_df)

final_df = pd.concat(final_df_list, axis = 0)

final_stages_df = pd.concat(final_stages_df_list, axis = 0)

In [ ]:
columns = final_stages_df.columns[~final_stages_df.columns.isin(['date'])].values

In [ ]:
columns

In [ ]:
pd.concat([final_stages_df[columns] + 2, final_stages_df[['date']]], axis = 1)

In [ ]:

# Export file to csv
final_df.to_csv('minuteSleep' + '.csv', index = False)
final_stages_df.to_csv('minutesStagesSleep' + '.csv', index = True)